# Membangun Model Regresi: Persiapkan dan Visualisasikan Data

## **Regresi Linear untuk Labu - Pelajaran 2**
#### Pendahuluan

Sekarang setelah Anda memiliki alat yang diperlukan untuk mulai membangun model pembelajaran mesin menggunakan Tidymodels dan Tidyverse, Anda siap untuk mulai mengajukan pertanyaan tentang data Anda. Saat bekerja dengan data dan menerapkan solusi ML, sangat penting untuk memahami cara mengajukan pertanyaan yang tepat agar dapat memanfaatkan potensi dataset Anda dengan baik.

Dalam pelajaran ini, Anda akan belajar:

-   Cara mempersiapkan data Anda untuk membangun model.

-   Cara menggunakan `ggplot2` untuk visualisasi data.

Pertanyaan yang perlu Anda jawab akan menentukan jenis algoritma ML yang akan Anda gunakan. Dan kualitas jawaban yang Anda dapatkan sangat bergantung pada sifat data Anda.

Mari kita lihat ini melalui latihan praktis.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>


<!--![Karya seni oleh \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Karya seni oleh \@allison_horst-->


## 1. Mengimpor data labu dan memanggil Tidyverse

Kita akan membutuhkan paket-paket berikut untuk mempelajari pelajaran ini:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) adalah [koleksi paket R](https://www.tidyverse.org/packages) yang dirancang untuk membuat ilmu data lebih cepat, mudah, dan menyenangkan!

Anda dapat menginstalnya dengan:

`install.packages(c("tidyverse"))`

Script di bawah ini memeriksa apakah Anda memiliki paket yang diperlukan untuk menyelesaikan modul ini dan menginstalnya untuk Anda jika ada yang belum terpasang.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Sekarang, mari kita jalankan beberapa paket dan muat [data](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) yang disediakan untuk pelajaran ini!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Sebuah `glimpse()` cepat langsung menunjukkan bahwa terdapat nilai kosong dan campuran antara string (`chr`) dan data numerik (`dbl`). `Date` memiliki tipe karakter, dan ada juga kolom aneh bernama `Package` di mana datanya merupakan campuran antara `sacks`, `bins`, dan nilai lainnya. Data ini, sebenarnya, cukup berantakan 😤.

Memang, tidak umum mendapatkan dataset yang sepenuhnya siap digunakan untuk membuat model ML langsung tanpa persiapan. Tapi jangan khawatir, dalam pelajaran ini, Anda akan belajar bagaimana mempersiapkan dataset mentah menggunakan pustaka standar R 🧑‍🔧. Anda juga akan mempelajari berbagai teknik untuk memvisualisasikan data. 📈📊
<br>

> Pengingat: Operator pipe (`%>%`) melakukan operasi dalam urutan logis dengan meneruskan sebuah objek ke depan ke dalam fungsi atau ekspresi panggilan. Anda dapat menganggap operator pipe seperti mengatakan "dan kemudian" dalam kode Anda.


## 2. Periksa data yang hilang

Salah satu masalah paling umum yang harus dihadapi oleh ilmuwan data adalah data yang tidak lengkap atau hilang. R merepresentasikan nilai yang hilang, atau tidak diketahui, dengan nilai sentinel khusus: `NA` (Not Available).

Jadi, bagaimana kita mengetahui bahwa data frame mengandung nilai yang hilang?
<br>
-   Salah satu cara yang sederhana adalah menggunakan fungsi dasar R `anyNA` yang mengembalikan objek logis `TRUE` atau `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Bagus, sepertinya ada beberapa data yang hilang! Itu adalah tempat yang baik untuk memulai.

-   Cara lain adalah dengan menggunakan fungsi `is.na()` yang menunjukkan elemen kolom mana yang hilang dengan nilai logis `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Oke, pekerjaan selesai, tetapi dengan data frame besar seperti ini, akan tidak efisien dan hampir mustahil untuk meninjau semua baris dan kolom satu per satu😴.

-   Cara yang lebih intuitif adalah dengan menghitung jumlah nilai yang hilang untuk setiap kolom:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Jauh lebih baik! Ada data yang hilang, tetapi mungkin itu tidak akan menjadi masalah untuk tugas ini. Mari kita lihat apa yang akan dihasilkan analisis lebih lanjut.

> Selain kumpulan paket dan fungsi yang luar biasa, R memiliki dokumentasi yang sangat baik. Sebagai contoh, gunakan `help(colSums)` atau `?colSums` untuk mengetahui lebih lanjut tentang fungsi tersebut.


## 3. Dplyr: Tata Bahasa Manipulasi Data

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Karya seni oleh @allison_horst</figcaption>


<!--![Karya seni oleh \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Karya seni oleh \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), sebuah paket dalam Tidyverse, adalah tata bahasa manipulasi data yang menyediakan serangkaian kata kerja konsisten untuk membantu Anda menyelesaikan tantangan manipulasi data yang paling umum. Pada bagian ini, kita akan menjelajahi beberapa kata kerja dari dplyr!


#### dplyr::select()

`select()` adalah fungsi dalam paket `dplyr` yang membantu Anda memilih kolom untuk disimpan atau dikecualikan.

Untuk membuat data frame Anda lebih mudah digunakan, hapus beberapa kolomnya menggunakan `select()`, dan simpan hanya kolom yang Anda butuhkan.

Sebagai contoh, dalam latihan ini, analisis kita akan melibatkan kolom `Package`, `Low Price`, `High Price`, dan `Date`. Mari kita pilih kolom-kolom tersebut.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` adalah fungsi dalam paket `dplyr` yang membantu Anda membuat atau memodifikasi kolom, sambil tetap mempertahankan kolom yang sudah ada.

Struktur umum dari `mutate` adalah:

`data %>%   mutate(new_column_name = what_it_contains)`

Mari kita coba menggunakan `mutate` pada kolom `Date` dengan melakukan operasi berikut:

1. Mengonversi tanggal (yang saat ini bertipe karakter) ke format bulan (ini adalah tanggal AS, jadi formatnya adalah `MM/DD/YYYY`).

2. Mengekstrak bulan dari tanggal ke kolom baru.

Di R, paket [lubridate](https://lubridate.tidyverse.org/) mempermudah bekerja dengan data Date-time. Jadi, mari kita gunakan `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` dan lihat bagaimana cara mencapai tujuan di atas. Kita dapat menghapus kolom Date karena tidak akan digunakan lagi dalam operasi berikutnya.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Woohoo! 🤩

Selanjutnya, mari kita buat kolom baru `Price`, yang mewakili harga rata-rata sebuah labu. Sekarang, mari kita ambil rata-rata dari kolom `Low Price` dan `High Price` untuk mengisi kolom Price yang baru.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Tapi tunggu!", Anda akan berkata setelah melihat seluruh data dengan `View(pumpkins)`, "Ada sesuatu yang aneh di sini!"🤔

Jika Anda melihat kolom `Package`, labu dijual dalam berbagai konfigurasi. Beberapa dijual dalam ukuran `1 1/9 bushel`, beberapa dalam ukuran `1/2 bushel`, beberapa per labu, beberapa per pon, dan beberapa dalam kotak besar dengan lebar yang bervariasi.

Mari kita verifikasi ini:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Luar Biasa!👏

Labu tampaknya sangat sulit untuk ditimbang secara konsisten, jadi mari kita menyaringnya dengan memilih hanya labu yang memiliki string *bushel* di kolom `Package` dan memasukkannya ke dalam kerangka data baru `new_pumpkins`.


#### dplyr::filter() dan stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): membuat subset data yang hanya berisi **baris** yang memenuhi kondisi Anda, dalam hal ini, labu dengan string *bushel* di kolom `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): mendeteksi keberadaan atau ketiadaan pola dalam sebuah string.

Paket [`stringr`](https://github.com/tidyverse/stringr) menyediakan fungsi sederhana untuk operasi string yang umum.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Anda dapat melihat bahwa kami telah mempersempitnya menjadi sekitar 415 baris data yang berisi labu per keranjang.🤩
<br>


#### dplyr::case_when()

**Tapi tunggu! Masih ada satu hal lagi yang perlu dilakukan**

Apakah Anda memperhatikan bahwa jumlah bushel berbeda di setiap baris? Anda perlu menormalkan harga sehingga Anda menampilkan harga per bushel, bukan per 1 1/9 atau 1/2 bushel. Saatnya melakukan beberapa perhitungan untuk menstandarkan ini.

Kita akan menggunakan fungsi [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) untuk *memodifikasi* kolom Price berdasarkan beberapa kondisi. `case_when` memungkinkan Anda memvektorisasi beberapa pernyataan `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Sekarang, kita bisa menganalisis harga per unit berdasarkan ukuran bushel mereka. Semua studi tentang bushel labu ini, bagaimanapun, menunjukkan betapa `pentingnya` untuk `memahami sifat data Anda`!

> ✅ Menurut [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), berat bushel tergantung pada jenis hasil panen, karena ini adalah pengukuran volume. "Satu bushel tomat, misalnya, seharusnya memiliki berat 56 pon... Daun dan sayuran hijau mengambil lebih banyak ruang dengan berat yang lebih sedikit, sehingga satu bushel bayam hanya memiliki berat 20 pon." Semuanya cukup rumit! Mari kita tidak repot-repot membuat konversi bushel ke pon, dan sebagai gantinya menetapkan harga berdasarkan bushel. Semua studi tentang bushel labu ini, bagaimanapun, menunjukkan betapa pentingnya untuk memahami sifat data Anda!
>
> ✅ Apakah Anda memperhatikan bahwa labu yang dijual per setengah bushel sangat mahal? Bisakah Anda mencari tahu alasannya? Petunjuk: labu kecil jauh lebih mahal daripada yang besar, mungkin karena ada jauh lebih banyak labu kecil per bushel, mengingat ruang kosong yang tidak terpakai yang diambil oleh satu labu pai besar yang berongga.


Sekarang terakhir, demi petualangan semata 💁‍♀️, mari kita pindahkan kolom Bulan ke posisi pertama yaitu `sebelum` kolom `Package`.

`dplyr::relocate()` digunakan untuk mengubah posisi kolom.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Kerja bagus!👌 Sekarang Anda memiliki dataset yang bersih dan rapi untuk membangun model regresi baru Anda!


## 4. Visualisasi data dengan ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografis oleh Dasani Madipalli</figcaption>


<!--![Infografis oleh Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Ada sebuah *pepatah bijak* yang berbunyi seperti ini:

> "Grafik sederhana telah memberikan lebih banyak informasi kepada pikiran analis data dibandingkan alat lainnya." --- John Tukey

Bagian dari peran seorang ilmuwan data adalah menunjukkan kualitas dan sifat data yang mereka kerjakan. Untuk melakukan ini, mereka sering membuat visualisasi yang menarik, seperti plot, grafik, dan diagram, yang menunjukkan berbagai aspek data. Dengan cara ini, mereka dapat secara visual menunjukkan hubungan dan celah yang sulit ditemukan dengan cara lain.

Visualisasi juga dapat membantu menentukan teknik pembelajaran mesin yang paling sesuai untuk data tersebut. Sebagai contoh, scatterplot yang tampak mengikuti garis menunjukkan bahwa data tersebut cocok untuk latihan regresi linier.

R menawarkan beberapa sistem untuk membuat grafik, tetapi [`ggplot2`](https://ggplot2.tidyverse.org/index.html) adalah salah satu yang paling elegan dan paling serbaguna. `ggplot2` memungkinkan Anda menyusun grafik dengan **menggabungkan komponen independen**.

Mari kita mulai dengan scatter plot sederhana untuk kolom Price dan Month.

Dalam kasus ini, kita akan mulai dengan [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), memberikan dataset dan pemetaan estetika (dengan [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) lalu menambahkan lapisan (seperti [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) untuk scatter plot.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Apakah ini plot yang berguna 🤷? Apakah ada yang mengejutkan Anda tentang ini?

Ini tidak terlalu berguna karena hanya menampilkan data Anda sebagai sebaran titik dalam bulan tertentu.  
<br>


### **Bagaimana cara membuatnya berguna?**

Untuk menampilkan grafik dengan data yang bermanfaat, biasanya Anda perlu mengelompokkan data dengan cara tertentu. Misalnya, dalam kasus kita, menemukan harga rata-rata labu untuk setiap bulan akan memberikan wawasan lebih tentang pola yang mendasari data kita. Hal ini membawa kita ke satu lagi fitur **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Agregasi terkelompok dalam R dapat dengan mudah dihitung menggunakan

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` mengubah unit analisis dari keseluruhan dataset menjadi kelompok individu seperti per bulan.

-   `dplyr::summarize()` membuat data frame baru dengan satu kolom untuk setiap variabel pengelompokan dan satu kolom untuk setiap statistik ringkasan yang telah Anda tentukan.

Sebagai contoh, kita dapat menggunakan `dplyr::group_by() %>% summarize()` untuk mengelompokkan labu berdasarkan kolom **Month** dan kemudian menemukan **harga rata-rata** untuk setiap bulan.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Ringkas!✨

Fitur kategorikal seperti bulan lebih baik direpresentasikan menggunakan plot batang 📊. Lapisan yang bertanggung jawab untuk diagram batang adalah `geom_bar()` dan `geom_col()`. Lihat `?geom_bar` untuk mengetahui lebih lanjut.

Mari kita buat satu!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩Ini adalah visualisasi data yang lebih berguna! Tampaknya menunjukkan bahwa harga tertinggi untuk labu terjadi pada bulan September dan Oktober. Apakah itu sesuai dengan ekspektasimu? Mengapa atau mengapa tidak?

Selamat telah menyelesaikan pelajaran kedua 👏! Kamu telah mempersiapkan data untuk membangun model, lalu menemukan lebih banyak wawasan menggunakan visualisasi!



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemah manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
